In [45]:
import os
from twilio.rest import Client
from twilio_config import TWILIO_ACCOUNT_SID,TWILIO_AUTH_TOKEN,PHONE_NUMBER,API_KEY_WAPI,K_NUMBER,J_NUMBER
import time
from requests import Request, Session
from requests.exceptions import ConnectionError, Timeout, TooManyRedirects
import json
import pandas as pd
import requests
from tqdm import tqdm
from datetime import datetime
from utils import request_wapi,get_forecast,create_df,send_message,get_date

In [46]:
query = 'La estrella'
api_key = API_KEY_WAPI

url = 'http://api.weatherapi.com/v1/forecast.json?key={}&q={}&days=1&aqi=no&alerts=no'.format(api_key,query)

In [47]:
response = requests.get(url).json()

In [48]:
response.keys()

dict_keys(['location', 'current', 'forecast'])

In [49]:
response['forecast']['forecastday'][0].keys()

dict_keys(['date', 'date_epoch', 'day', 'astro', 'hour'])

In [50]:
len(response['forecast']['forecastday'][0]['hour'])

24

In [51]:
response['forecast']['forecastday'][0]['hour'][1]['time'].split()[0]

'2023-06-22'

In [52]:
response['forecast']['forecastday'][0]['hour'][1]['time'].split()[1].split(':')[0]

'01'

In [53]:
response['forecast']['forecastday'][0]['hour'][0]['condition']['text']

'Patchy rain possible'

In [54]:
response['forecast']['forecastday'][0]['hour'][0]['temp_c']

12.6

In [55]:
response['forecast']['forecastday'][0]['hour'][0]['will_it_rain']

1

In [56]:
response['forecast']['forecastday'][0]['hour'][0]['chance_of_rain']

77

## Dataframe

In [57]:
def get_forecast(response,i):

    fecha = response['forecast']['forecastday'][0]['hour'][i]['time'].split()[0]
    hora = response['forecast']['forecastday'][0]['hour'][i]['time'].split()[1].split(':')[0]
    condicion = response['forecast']['forecastday'][0]['hour'][i]['condition']['text']
    temp = response['forecast']['forecastday'][0]['hour'][i]['temp_c']
    rain = response['forecast']['forecastday'][0]['hour'][i]['will_it_rain']
    rain_prov = response['forecast']['forecastday'][0]['hour'][i]['chance_of_rain']
    
    return fecha,hora,condicion,temp,rain,rain_prov

In [58]:
datos = []

for i in tqdm(range(len(response['forecast']['forecastday'][0]['hour'])), colour = 'green'):

    datos.append(get_forecast(response,i))

100%|██████████| 24/24 [00:00<00:00, 23984.58it/s]


In [59]:
datos

[('2023-06-22', '00', 'Patchy rain possible', 12.6, 1, 77),
 ('2023-06-22', '01', 'Patchy rain possible', 12.4, 1, 79),
 ('2023-06-22', '02', 'Partly cloudy', 12.2, 0, 0),
 ('2023-06-22', '03', 'Mist', 12.1, 0, 0),
 ('2023-06-22', '04', 'Mist', 12.0, 0, 0),
 ('2023-06-22', '05', 'Mist', 12.0, 0, 0),
 ('2023-06-22', '06', 'Mist', 11.9, 0, 0),
 ('2023-06-22', '07', 'Sunny', 16.3, 0, 0),
 ('2023-06-22', '08', 'Sunny', 19.5, 0, 0),
 ('2023-06-22', '09', 'Sunny', 22.2, 0, 0),
 ('2023-06-22', '10', 'Patchy rain possible', 23.4, 1, 77),
 ('2023-06-22', '11', 'Patchy rain possible', 24.8, 1, 78),
 ('2023-06-22', '12', 'Patchy rain possible', 27.1, 0, 60),
 ('2023-06-22', '13', 'Patchy rain possible', 27.5, 1, 71),
 ('2023-06-22', '14', 'Patchy rain possible', 26.9, 1, 86),
 ('2023-06-22', '15', 'Sunny', 26.9, 0, 0),
 ('2023-06-22', '16', 'Patchy rain possible', 24.6, 0, 62),
 ('2023-06-22', '17', 'Sunny', 22.0, 0, 0),
 ('2023-06-22', '18', 'Patchy rain possible', 18.0, 0, 60),
 ('2023-06-22', 

In [60]:
col = ['Fecha','Hora','Condicion','Temp','Rain','Rain_prov']
df = pd.DataFrame(datos,columns=col)

In [61]:
df

,Fecha,Hora,Condicion,Temp,Rain,Rain_prov
0,2023-06-22,00,Patchy rain possible,12.6,1,77
1,2023-06-22,01,Patchy rain possible,12.4,1,79
2,2023-06-22,02,Partly cloudy,12.2,0,0
3,2023-06-22,03,Mist,12.1,0,0
4,2023-06-22,04,Mist,12.0,0,0
5,2023-06-22,05,Mist,12.0,0,0
6,2023-06-22,06,Mist,11.9,0,0
7,2023-06-22,07,Sunny,16.3,0,0
8,2023-06-22,08,Sunny,19.5,0,0
9,2023-06-22,09,Sunny,22.2,0,0


In [62]:
df_rain = df[(df['Rain'] == 1)]
df_redu = df_rain[['Hora','Temp','Condicion']]
df_redu.set_index('Hora',inplace = True)

In [63]:
from datetime import datetime

datetime.now().time().hour

13

In [64]:
df_temp = df[['Hora','Temp','Condicion']]


In [65]:
current_hour = str(datetime.now().time().hour)
if len(current_hour) == 1:
    current_hour = current_hour.zfill(2)

df_temp = df_temp[df_temp['Hora'] == current_hour]

In [66]:
datetime.now().time().hour

13

In [67]:
df_temp

,Hora,Temp,Condicion
13,13,27.5,Patchy rain possible


In [68]:
df_temp.set_index('Hora',inplace = True)

In [69]:
df_redu

,Temp,Condicion
Hora,,
00,12.6,Patchy rain possible
01,12.4,Patchy rain possible
10,23.4,Patchy rain possible
11,24.8,Patchy rain possible
13,27.5,Patchy rain possible
14,26.9,Patchy rain possible
19,15.7,Patchy rain possible
20,14.0,Light rain shower
21,13.9,Light rain shower


In [70]:

if df_redu.empty:
    print('no hay plan arrunchis, no va a llover :C')
else:
    print('wuju')

wuju


In [71]:
df_redu

,Temp,Condicion
Hora,,
00,12.6,Patchy rain possible
01,12.4,Patchy rain possible
10,23.4,Patchy rain possible
11,24.8,Patchy rain possible
13,27.5,Patchy rain possible
14,26.9,Patchy rain possible
19,15.7,Patchy rain possible
20,14.0,Light rain shower
21,13.9,Light rain shower


In [72]:
k = K_NUMBER
j = J_NUMBER

## Mensaje SMS desde twilio

In [73]:
# Download the helper library from https://www.twilio.com/docs/python/install
import os
from twilio.rest import Client


In [74]:
time.sleep(2)
account_sid = TWILIO_ACCOUNT_SID
auth_token = TWILIO_AUTH_TOKEN

In [75]:
'Hoy va a llover, abrígate bien :) \n\n\n ' + str(df_redu)


'Hoy va a llover, abrígate bien :) \n\n\n       Temp             Condicion\nHora                            \n00    12.6  Patchy rain possible\n01    12.4  Patchy rain possible\n10    23.4  Patchy rain possible\n11    24.8  Patchy rain possible\n13    27.5  Patchy rain possible\n14    26.9  Patchy rain possible\n19    15.7  Patchy rain possible\n20    14.0     Light rain shower\n21    13.9     Light rain shower\n22    13.8     Light rain shower\n23    13.5     Light rain shower'

In [76]:


# Find your Account SID and Auth Token at twilio.com/console
# and set the environment variables. See http://twil.io/secure

client = Client(account_sid, auth_token)
if df_redu.empty:
    template = '\n Hoy no hay arrunchis, no va a llover :c \n\n\n ' + str(df_redu) 
else:
    template = '\n Hoy va a llover en '+ query +', abrígate bien :) \n\n\n ' + str(df_temp)

message = client.messages \
    .create(
        
        body=template,
        from_=PHONE_NUMBER,
        to=k
     )

print('Mensaje enviado: ',message.sid, template)



Mensaje enviado:  SM12dea92f9d66ce382c6362e81a4d28e0 
 Hoy va a llover en La estrella, abrígate bien :) 


       Temp             Condicion
Hora                            
13    27.5  Patchy rain possible


In [77]:
'Hoy va a llover en '+ query +', abrígate bien :) \n\n\n ' + str(df_temp)

'Hoy va a llover en La estrella, abrígate bien :) \n\n\n       Temp             Condicion\nHora                            \n13    27.5  Patchy rain possible'

In [78]:
df_temp

,Temp,Condicion
Hora,,
13,27.5,Patchy rain possible


In [79]:
df

,Fecha,Hora,Condicion,Temp,Rain,Rain_prov
0,2023-06-22,00,Patchy rain possible,12.6,1,77
1,2023-06-22,01,Patchy rain possible,12.4,1,79
2,2023-06-22,02,Partly cloudy,12.2,0,0
3,2023-06-22,03,Mist,12.1,0,0
4,2023-06-22,04,Mist,12.0,0,0
5,2023-06-22,05,Mist,12.0,0,0
6,2023-06-22,06,Mist,11.9,0,0
7,2023-06-22,07,Sunny,16.3,0,0
8,2023-06-22,08,Sunny,19.5,0,0
9,2023-06-22,09,Sunny,22.2,0,0
